In [1]:
!pip install loralib

In [2]:
import pandas as pd
import gc
import numpy as np
import os

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
from peft import LoraConfig, get_peft_model
import loralib as lora

In [5]:
import torch
from torch.utils.data import Dataset, DataLoader

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
df_task_1_train=pd.read_csv('/content/drive/MyDrive/df_task_1_train.csv', index_col=False)

In [8]:
# df_task_1_test=pd.read_csv('/content/drive/MyDrive/df_task_1_test.csv', index_col=False)

In [9]:
df_task_1_train.head()

,Query Item Pair,soft_target
0,Query: !awnmower tires without rims Product Ti...,0.0
1,Query: !awnmower tires without rims Product Ti...,1.0
2,Query: !awnmower tires without rims Product Ti...,1.0
3,Query: !awnmower tires without rims Product Ti...,0.0
4,Query: # 10 self-seal envelopes without window...,1.0


In [10]:
print(df_task_1_train['Query Item Pair'].iloc[1])

Query: !awnmower tires without rims Product Title: (Set of 2) 15x6.00-6 Husqvarna/Poulan Tire Wheel Assy .75" Bearing Product Description: No fuss. Just take off your old assembly and replace with this. No need for tubes or tire shops. No messing around with mounting these yourself. Perfect replacement for your machine. Will make your machine look and feel new. **** .75 precision ball bearings ****. Product Bullet Point: Tire size:15x6.00-6 Ply: 4 Tubeless
6x4.5 Wheel with 3/4" Precision bearings; Hub is 3" Long with .75" precision ball bearings. No grease required. Color: Husqvarna Silver
Husqvarna wheel number: 532106732 replaces 106732x645, 106732x643, 106732x417, 532141446, 532138336, 5321383-36 532125102; Husqvarna tire number 5321122073
ATW-001
Tire OD: 14.96; Tire SW: 6.3; PSI: 30; Max Load: 570 lbs. Product Brand: Antego Tire & Wheel Product Color: Husqvarna Silver


In [11]:
#df_task_1_test.head()

In [12]:
class QueryItemDataset(Dataset):
  def __init__(self, df, query_col, target_col, tokenizer, max_length=512):
    self.texts=df[query_col].tolist()
    self.labels = df[target_col].tolist()
    self.tokenizer = tokenizer
    self.max_length = max_length

  def __len__(self):
    return len(self.texts)

  def __getitem__(self, idx):
    text = self.texts[idx]
    label = self.labels[idx]

    encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

    return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "labels": torch.tensor(label, dtype=torch.float),
        }


In [13]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, pipeline, AutoConfig, AutoModelForCausalLM, AutoModel

In [14]:
class LlamaRelevanceScore(nn.Module):
    def __init__(self, model_name, mlp_hidden_dim=1024):
        super().__init__()

        self.llama = AutoModel.from_pretrained(
            model_name,
            torch_dtype=torch.bfloat16
            )

        hidden_size = self.llama.config.hidden_size

        #logit
        self.score_head = nn.Sequential(
            nn.Linear(hidden_size, mlp_hidden_dim),
            nn.ReLU(),
            nn.Linear(mlp_hidden_dim, 1)
        )

    def forward(self,
    input_ids=None,
    attention_mask=None,
    inputs_embeds=None,
    position_ids=None,
    past_key_values=None,
    labels=None,
                **kwargs):
        outputs = self.llama(
        input_ids=input_ids,
        attention_mask=attention_mask,
        inputs_embeds=inputs_embeds,
        position_ids=position_ids,
        past_key_values=past_key_values,
        **kwargs
        )

        last_token = outputs.last_hidden_state[:, -1, :]

        logits = self.score_head(last_token).squeeze(-1)

        loss = None
        if labels is not None:
            loss_fn = nn.BCEWithLogitsLoss()
            loss = loss_fn(logits, labels.float())

        return {
            "loss": loss,
            "logits": logits,
        }


In [15]:
model_id = "meta-llama/Llama-3.2-1B"

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
def getLoraModel(model):
    # Adjusted LoRA Config with r=128 instead of 256 mentioned in paper
    lora_config = LoraConfig(
        r=128,
        target_modules=["q_proj", "v_proj"],
        lora_dropout=0.05,
        task_type="FEATURE_EXTRACTION",
        lora_alpha=64

    )

    model = get_peft_model(model, lora_config)
    lora.mark_only_lora_as_trainable(model)
    model.print_trainable_parameters()

    return model

In [17]:
model = LlamaRelevanceScore(model_name=model_id)
model.score_head = model.score_head.to(torch.bfloat16)
model.eval()

`torch_dtype` is deprecated! Use `dtype` instead!


LlamaRelevanceScore(
  (llama): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
    

In [ ]:
model = getLoraModel(model)

trainable params: 13,631,488 || all params: 1,251,545,089 || trainable%: 1.0892


In [19]:
query_col='Query Item Pair'
target_col='soft_target'
dataset = QueryItemDataset(df_task_1_train, query_col, target_col, tokenizer)
dataloader = DataLoader(dataset, batch_size=43, shuffle=True)

In [20]:
print(dataloader)

In [ ]:
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm


lr = 1e-5
num_epochs = 3
gradient_accumulation_steps = 1


optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=lr,
    weight_decay=0.01
)


num_training_steps = num_epochs * len(dataloader)
num_warmup_steps = int(0.1 * num_training_steps) 

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=num_training_steps
)


In [22]:
import os

BASE_CKPT_DIR = "/content/drive/MyDrive/llama_relevance_checkpoints"
os.makedirs(BASE_CKPT_DIR, exist_ok=True)


In [ ]:
save_every = max(1, int(0.2 * len(dataloader)))

def save_checkpoint(model, optimizer, scheduler, epoch, step):
    ckpt_dir = os.path.join(
        BASE_CKPT_DIR, f"epoch{epoch}_step{step}"
    )
    os.makedirs(ckpt_dir, exist_ok=True)

  
    model.save_pretrained(ckpt_dir)

   
    torch.save(
        model.score_head.state_dict(),
        os.path.join(ckpt_dir, "score_head.pt")
    )

   
    torch.save(
        {
            "optimizer": optimizer.state_dict(),
            "scheduler": scheduler.state_dict(),
            "epoch": epoch,
            "step": step,
        },
        os.path.join(ckpt_dir, "trainer_state.pt")
    )

    print(f"Saved checkpoint at {ckpt_dir}")


In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cuda


In [25]:
torch.cuda.empty_cache()
gc.collect()

150

In [ ]:
model.train()
model.to(device)
global_step = 0

for epoch in range(num_epochs):
    epoch_loss = 0.0
    progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1}")

    for step, batch in enumerate(progress_bar):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

        loss = outputs["loss"]
        loss = loss / gradient_accumulation_steps
        loss.backward()

        if (step + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
            global_step += 1

        epoch_loss += loss.item()
        progress_bar.set_postfix(
            loss=loss.item(),
            lr=scheduler.get_last_lr()[0]
        )

        # checkpointing every 20% epoch
        if (step + 1) % save_every == 0:
            save_checkpoint(
                model=model,
                optimizer=optimizer,
                scheduler=scheduler,
                epoch=epoch + 1,
                step=step + 1,
            )

    avg_epoch_loss = epoch_loss / len(dataloader)
    print(f"\nEpoch {epoch+1} finished | Avg loss: {avg_epoch_loss:.4f}\n")

Epoch 1:  20%|█▉        | 815/4078 [12:52<59:04,  1.09s/it, loss=0.666, lr=6.66e-6]

Saved checkpoint at /content/drive/MyDrive/llama_relevance_checkpoints/epoch1_step815


Epoch 1:  40%|███▉      | 1630/4078 [25:44<43:21,  1.06s/it, loss=0.649, lr=9.63e-6]

Saved checkpoint at /content/drive/MyDrive/llama_relevance_checkpoints/epoch1_step1630


Epoch 1:  60%|█████▉    | 2445/4078 [38:35<28:51,  1.06s/it, loss=0.691, lr=8.89e-6]

...

Saved checkpoint at /content/drive/MyDrive/llama_relevance_checkpoints/epoch3_step3260


Epoch 3: 100%|█████████▉| 4075/4078 [1:04:18<00:03,  1.08s/it, loss=0.474, lr=2.72e-9]

Saved checkpoint at /content/drive/MyDrive/llama_relevance_checkpoints/epoch3_step4075


Epoch 3: 100%|██████████| 4078/4078 [1:04:20<00:00,  1.06it/s, loss=0.716, lr=0]


Epoch 3 finished | Avg loss: 0.5760

